In [86]:
import torch
from torch import nn

In [87]:
channels = torch.tensor([0, 1], dtype=torch.float32)
normal_matrix1 = torch.randn(4, 4)
normal_matrix2 = torch.randn(4, 4)
matrix = torch.stack([normal_matrix1,
                     normal_matrix2] )

In [88]:

##定义卷积层
class Convol_2(nn.Module):
    def __init__(self , out_channels , in_channels , kernel_sizex , kernel_sizey):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_sizex , self.kernel_sizey = kernel_sizex , kernel_sizey # kernel_size为 (x,y) 数对
        self.weight = nn.Parameter(torch.randn( out_channels , in_channels , kernel_sizex , kernel_sizey ) )
        ##randn 生成标准正态分布数据
        # self.bias = nn.Parameter(torch.randn(out_channels) )
    '''    
    def forward(self,X):
        return F.conv2d(X,self.weight,self.bias,padding = 1)
    '''
    
    ##具体实现

    ## 2d实现(直接卷积)
    def corr2d(self,X,K):
        h , w = K.shape
        Y = torch.zeros(X.shape[0] - h + 1,X.shape[1] - w + 1)
        for i in range(Y.shape[0]):
            for j in range(Y.shape[1]):
                Y[i,j] = (X[i:i+h,j:j+w] * K).sum() ## * 代表点积 ，即每个对应元素相乘
        return Y
    
    def multi_in_channels(self,X,K):
        #错误示范
        '''
        return torch.sum(torch.stack([corr2d(X, k) for k in K]), dim = 0) # stack 对 最外层，也就是shape[0]进行遍历
        #return torch.sum(torch.stack([corr2d(X,k) for k in K])) # k是遍历K的最外层取出的矩阵，n-1维度
        '''
        #正确示范
        return sum(self.corr2d(x, k) for x, k in   zip(X, K)) # zip (X,K) X,K外面的维度做遍历

        #zip(X,K) 将第一维度配对，迭代返回X,K的配对后两维数据

    def multi_out_channels(self,X,K):
        return torch.stack([self.multi_in_channels(X,k) for k in K],dim = 0)
        #不加sum的话，所有结果都叠加起来
        #因为输出通道不用叠加

    def forward(self,X):
        # return self.multi_out_channels(X,self.weight)+self.bias.reshape(self.out_channels,1,1)
        return self.multi_out_channels(X,self.weight)

def corr2d(X,K):
    h , w = K.shape
    Y = torch.zeros(X.shape[0] - h + 1,X.shape[1] - w + 1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h,j:j+w] * K).sum() ## * 代表点积 ，即每个对应元素相乘
    return Y
def multi_in_channels(X,K):
    return sum(corr2d(x, k) for x, k in   zip(X, K)) # zip (X,K) X,K外面的维度做遍历
def multi_out_channels(X,K):
    return torch.stack([multi_in_channels(X,k) for k in K],dim = 0)

In [89]:
W = torch.ones(2,1,2,2)
print(multi_out_channels(matrix,W))
Y = multi_out_channels(matrix,W)

tensor([[[-1.2695, -0.9194,  1.5498],
         [-2.2975, -1.7072, -0.5453],
         [-1.0065, -1.5351, -1.1163]],

        [[-1.2695, -0.9194,  1.5498],
         [-2.2975, -1.7072, -0.5453],
         [-1.0065, -1.5351, -1.1163]]])


In [90]:
myconv = Convol_2(2,1,2,2)

In [91]:
for i in range(1000):
    Y_hat = myconv(matrix)
    l = (Y_hat - Y)**2 # loss
    myconv.zero_grad() #清零梯度
    l.sum().backward() # 记录梯度
    myconv.weight.data[:] -= 5e-4 * myconv.weight.grad # 学习率乘以 -grad,每一个weight data都是
    if (i + 1) % 20 == 0:
        print(f'batch {i+1}, loss {l.sum():.3f}')

batch 20, loss 70.444
batch 40, loss 49.325
batch 60, loss 35.954
batch 80, loss 27.149
batch 100, loss 21.104
batch 120, loss 16.780
batch 140, loss 13.571
batch 160, loss 11.113
batch 180, loss 9.182
batch 200, loss 7.635
batch 220, loss 6.377
batch 240, loss 5.344
batch 260, loss 4.489
batch 280, loss 3.777
batch 300, loss 3.182
batch 320, loss 2.684
batch 340, loss 2.265
batch 360, loss 1.913
batch 380, loss 1.617
batch 400, loss 1.367
batch 420, loss 1.157
batch 440, loss 0.979
batch 460, loss 0.829
batch 480, loss 0.702
batch 500, loss 0.595
batch 520, loss 0.504
batch 540, loss 0.428
batch 560, loss 0.363
batch 580, loss 0.308
batch 600, loss 0.262
batch 620, loss 0.222
batch 640, loss 0.189
batch 660, loss 0.161
batch 680, loss 0.137
batch 700, loss 0.116
batch 720, loss 0.099
batch 740, loss 0.084
batch 760, loss 0.072
batch 780, loss 0.061
batch 800, loss 0.052
batch 820, loss 0.045
batch 840, loss 0.038
batch 860, loss 0.033
batch 880, loss 0.028
batch 900, loss 0.024
batch 